# <center>Analisando dados BIDMC PPG - Respiração</center>

Os dados originais (https://physionet.org/content/bidmc/1.0.0/) foram adquiridos de pacientes criticamente enfermos durante atendimento hospitalar no Beth Israel Deaconess Medical Center (Boston, MA, EUA). Dois anotadores anotaram manualmente respirações individuais em cada registro usando o sinal respiratório de impedância. As 53 gravações dentro do conjunto de dados, cada uma com 8 minutos de duração, cada uma contém:
<ul>
<li> Sinais fisiológicos, como PPG, sinal respiratório de impedância e eletrocardiograma (ECG). Eles são amostrados em 125 Hz; </li>
<li> Parâmetros fisiológicos, como frequência cardíaca (FC), frequência respiratória (FR) e nível de saturação de oxigênio no sangue (SpO2). Eles são amostrados em 1 Hz; </li>
<li> Parâmetros fixos, como idade e sexo; </li>
<li> Anotações manuais de respirações.</li>
</ul>

## Dados Formato CSV

Arquivos CSV separados são fornecidos para cada gravação (onde ## é o número do paciente), contendo:
<ul>
<li> bidmc _ ## _ Breaths.csv: anotações de respiração manual; </li>
<li> bidmc _ ## _ Signals.csv: sinais fisiológicos; </li>
<li> bidmc _ ## _ Numerics.csv: Parâmetros fisiológicos; </li>
<li> bidmc _ ## _ Fix.txt: variáveis fixas.</li>
</ul>

Artigos de referencia - (https://ieeexplore.ieee.org/document/6713498) - (https://ieeexplore.ieee.org/document/8648205) - https://ieeexplore.ieee.org/abstract/document/8410910

In [ ]:
%%time
import pandas as pd
import os
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import numpy  as np

from PyEMD import EEMD
from scipy.fft import fft, fftfreq
from datetime import datetime
from plotly.subplots import make_subplots

In [ ]:
os.chdir("C:\\Users\\ayalo\\PPG\\HTML" )

In [ ]:
caminho3 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_02_Numerics.csv"
caminho4 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_01_Fix.txt"
caminho5 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_01_Signals.csv"
pd.set_option("display.max_rows", None)

In [ ]:
# Inicializa o modo Plotly Notebook 
py.init_notebook_mode(connected=False)

# Função de Setup: devido ao Plotly estar sendo utilizado no Google Colab, 
# é preciso definir a função abaixo e chamá-la sempre que for exibir um gráfico
def configure_plotly_browser_state():
  import IPython
  from plotly.offline import init_notebook_mode 
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))
  init_notebook_mode(connected=False)

In [ ]:
%%time

#from numpy import empty
PadraoOuro = np.empty((54, 481))
ValorOuro = np.array(range(54))
Sinal_Original = np.empty((54, 60001))
Sinal_Original_IMP = np.empty((54, 60001))

for i in range(54):
    
    if i == 0:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print(current_time)
    if i > 0 and i < 10:
        caminho3 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_0"+str(i)+"_Numerics.csv"
        caminho5 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_0"+str(i)+"_Signals.csv"
        
        #Dados padrao ouro respiração
        numerics = pd.read_csv(caminho3)
        respiracao_signal = numerics[' RESP'].fillna(0)
        sinal_convertido = respiracao_signal.to_numpy()
        media = sinal_convertido.mean()
        PadraoOuro[i] = sinal_convertido
        ValorOuro[i] = media
        
        #Dados sinal Impedancia
        signals = pd.read_csv(caminho5)
        s = signals[[' RESP']]
        s = s.to_numpy()
        s_trans = s.transpose()
        Signal_IMP = s_trans[0]
        Sinal_Original_IMP[i] = Signal_IMP
        
        #Dados sinal PPG
        signals = pd.read_csv(caminho5)
        s = signals[[' PLETH']]
        s = s.to_numpy()
        s_trans = s.transpose()
        Signal_PPG = s_trans[0]
        Sinal_Original[i] = Signal_PPG
        
        
    if i >= 10 and i<54:
        caminho3 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_"+str(i)+"_Numerics.csv"
        caminho5 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_"+str(i)+"_Signals.csv"
        
        #Dados padrao ouro respiração
        numerics = pd.read_csv(caminho3)
        respiracao_signal = numerics[' RESP'].fillna(0)
        sinal_convertido = respiracao_signal.to_numpy()
        media = sinal_convertido.mean()
        PadraoOuro[i] = sinal_convertido
        ValorOuro[i] = media
        
        #Dados sinal Impedancia
        signals = pd.read_csv(caminho5)
        s = signals[[' RESP']]
        s = s.to_numpy()
        s_trans = s.transpose()
        Signal_IMP = s_trans[0]
        Sinal_Original_IMP[i] = Signal_IMP
        
        #Dados sinal PPG
        signals = pd.read_csv(caminho5)
        s = signals[[' PLETH']]
        s = s.to_numpy()
        s_trans = s.transpose()
        Signal_PPG = s_trans[0]
        Sinal_Original[i] = Signal_PPG
        

In [ ]:
## PADRAO OURO
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

# Create figure
fig = make_subplots(rows=1, cols=1)
X = numerics['Time [s]']
X = X[0:60]

# Add all traces of pacient
for step in range(1,54):
    Y = PadraoOuro[step]
    fig.add_scatter(x=X, y=Y[0:60],
                    row=1, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="black")

# Make 10th trace visible
fig.data[0].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}] 
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    steps=steps
)]

fig.update_layout(
    title="Frequencia respiratoria com sensor de impedancia:",
    xaxis_title="Segundos (s)",
    yaxis_title="Respirações (rpm)",
    sliders=sliders,
    plot_bgcolor='rgba(0,0,0,0)',
    height=400
)

# Edit slider labels
fig['layout']['sliders'][0]['currentvalue']['prefix']='Paciente: '
for i in range(0,53):
    fig['layout']['sliders'][0]['steps'][i]['label']=i+1


fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_11'+'.html',auto_open=False)

In [ ]:
%%time

## Sinal Impedancia e PPG
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

X = signals['Time [s]']
X = X[0:7500]

# Create figure
fig = make_subplots(rows=2, cols=1)
# Add all traces of pacient
for step in range(1,54):
    Y = Sinal_Original_IMP[step]
    fig.add_scatter(x=X, y=Y[0:7500],
                    row=1, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="gold")
for step in range(1,54):
    Y = Sinal_Original[step]
    fig.add_scatter(x=X, y=Y[0:7500],
                    row=2, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="black")

# Make 10th trace visible
fig.data[0].visible = True
fig.data[53].visible = True

# Create and add slider
steps = []
for i in range(53):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},],  
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+53] = True
    steps.append(step)

sliders = [dict(
    steps=steps
)]

# Update xaxis properties
fig.update_xaxes(title_text="segundos (s)", row=2, col=1)

# Update yaxis properties
fig.update_yaxes(title_text="IMPEDANCIA", row=1, col=1)
fig.update_yaxes(title_text="PPG", row=2, col=1)

fig.update_layout(
    title="Sinais de Impedancia e PPG:",
    sliders=sliders,
    plot_bgcolor='rgba(0,0,0,0)',
    height=600
)

# Edit slider labels
fig['layout']['sliders'][0]['currentvalue']['prefix']='Paciente: '
for i in range(0,53):
    fig['layout']['sliders'][0]['steps'][i]['label']=i+1


fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_12'+'.html',auto_open=False)


In [ ]:
%%time

#from numpy import empty
Sinal_Original_IMP = np.empty((54, 7500))
IMF1_IMP = np.empty((54, 7500))
IMF2_IMP = np.empty((54, 7500))
IMF3_IMP = np.empty((54, 7500))
IMF4_IMP = np.empty((54, 7500))
IMF5_IMP = np.empty((54, 7500))
IMF6_IMP = np.empty((54, 7500))
IMF7_IMP = np.empty((54, 7500))
IMF8_IMP = np.empty((54, 7500))

for i in range(54):
    
    if i == 0:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print(current_time)
    if i > 0 and i < 10:
        caminho5 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_0"+str(i)+"_Signals.csv"
        
        ####################################################
        # EEMD - IMPEDANCIA
        signals = pd.read_csv(caminho5)
        s = signals[[' RESP']]
        s = s.to_numpy()
        s_trans = s.transpose()
        Signal_IMP = s_trans[0]
        time = np.linspace(0, 1, 7500)

        # Assign max IMF on signal
        emd1 = EEMD()
        #emd1 = EEMD(MAX_ITERATION = 10)

        # Execute EEMD on signal
        #IMF = emd1.eemd(Signal_PPG,time, max_imf = 3)
        IMF = emd1.eemd(Signal_IMP[0:7500],time)

        Sinal_Original_IMP[i] = Signal_IMP[0:7500]
        IMF1_IMP[i] = IMF[0]
        IMF2_IMP[i] = IMF[1]
        IMF3_IMP[i] = IMF[2]
        IMF4_IMP[i] = IMF[3]
        IMF5_IMP[i] = IMF[4]
        IMF6_IMP[i] = IMF[5]
        IMF7_IMP[i] = IMF[6]
        IMF8_IMP[i] = IMF[6]
        
                
    if i >= 10 and i<54:
        caminho5 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_"+str(i)+"_Signals.csv"
        
        ####################################################
        # EEMD - IMPEDANCIA
        signals = pd.read_csv(caminho5)
        s = signals[[' RESP']]
        s = s.to_numpy()
        s_trans = s.transpose()
        Signal_IMP = s_trans[0]
        time = np.linspace(0, 1, 7500)

        # Assign max IMF on signal
        emd1 = EEMD()
        #emd1 = EEMD(MAX_ITERATION = 10)

        # Execute EEMD on signal
        #IMF = emd1.eemd(Signal_PPG,time, max_imf = 3)
        IMF = emd1.eemd(Signal_IMP[0:7500],time)

        Sinal_Original_IMP[i] = Signal_IMP[0:7500]
        IMF1_IMP[i] = IMF[0]
        IMF2_IMP[i] = IMF[1]
        IMF3_IMP[i] = IMF[2]
        IMF4_IMP[i] = IMF[3]
        IMF5_IMP[i] = IMF[4]
        IMF6_IMP[i] = IMF[5]
        IMF7_IMP[i] = IMF[6]
        IMF8_IMP[i] = IMF[7]
        
        ####################################################

In [ ]:
%%time

## IMF PLOT
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

X = signals['Time [s]']
X = X[0:2000]

# Create figure
fig = make_subplots(rows=9, cols=1)
# Add all traces of pacient

for step in range(1,54):
    Y = Sinal_Original_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=1, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="black")
for step in range(1,54):
    Y = IMF1_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=2, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF2_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=3, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF3_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=4, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF4_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=5, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF5_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=6, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF6_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=7, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF7_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=8, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF8_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=9, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")

# Make 10th trace visible
fig.data[0].visible = True
fig.data[53].visible = True
fig.data[106].visible = True
fig.data[159].visible = True
fig.data[212].visible = True
fig.data[265].visible = True
fig.data[318].visible = True
fig.data[371].visible = True
fig.data[424].visible = True

# Create and add slider
steps = []
for i in range(53):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Padrão ouro dos pacientes: " + str(i+1)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+53] = True
    step["args"][0]["visible"][i+106] = True
    step["args"][0]["visible"][i+159] = True
    step["args"][0]["visible"][i+212] = True
    step["args"][0]["visible"][i+265] = True
    step["args"][0]["visible"][i+318] = True
    step["args"][0]["visible"][i+371] = True
    step["args"][0]["visible"][i+424] = True

    steps.append(step)

sliders = [dict(    
    steps=steps
)]

# Update xaxis properties
fig.update_xaxes(title_text="segundos (s)", row=9, col=1)


# Update yaxis properties
fig.update_yaxes(title_text="IMPEDANCIA", row=1, col=1)
fig.update_yaxes(title_text="IMF1", row=2, col=1)
fig.update_yaxes(title_text="IMF2", row=3, col=1)
fig.update_yaxes(title_text="IMF3", row=4, col=1)
fig.update_yaxes(title_text="IMF4", row=5, col=1)
fig.update_yaxes(title_text="IMF5", row=6, col=1)
fig.update_yaxes(title_text="IMF6", row=7, col=1)
fig.update_yaxes(title_text="IMF7", row=8, col=1)
fig.update_yaxes(title_text="IMF8", row=9, col=1)

fig.update_layout(
    title="EEMD do sinal de Impedancia:",
    sliders=sliders,
    plot_bgcolor='rgba(0,0,0,0)',
    height=900
)

# Edit slider labels
fig['layout']['sliders'][0]['currentvalue']['prefix']='Paciente: '
for i in range(0,53):
    fig['layout']['sliders'][0]['steps'][i]['label']=i+1


fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_13'+'.html',auto_open=False)

In [ ]:
%%time

# Number of sample points
N = 7500
# sample spacing
T = 1.0 / 125

XfftIMFO_IMP = np.empty((54, 3750))
YfftIMFO_IMP = np.empty((54, 3750))
XfftIMFA_IMP = np.empty((54, 3750))
YfftIMFA_IMP = np.empty((54, 3750))
XfftIMFP_IMP = np.empty((54, 3750))
YfftIMFP_IMP = np.empty((54, 3750))
XfftIMFU_IMP = np.empty((54, 3750))
YfftIMFU_IMP = np.empty((54, 3750))
ValorMedidoIMFO_IMP = np.array(range(54))
ValorMedidoIMFA_IMP = np.array(range(54))
ValorMedidoIMFP_IMP = np.array(range(54))
ValorMedidoIMFU_IMP = np.array(range(54))

for i in range(54):
    
    if i == 0:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print(current_time)
    if i > 0 and i < 54:

        ####################################################
        # FFT

        y0 = Sinal_Original_IMP[i]
        yf0 = fft(y0)
        x0 = fftfreq(N, T)[:N//2]
        y0 = 2.0/N * np.abs(yf0[0:N//2])
        XfftIMFO_IMP[i] = x0
        YfftIMFO_IMP[i] = y0
        
        y1 = IMF6_IMP[i]
        yf1 = fft(y1)
        x1 = fftfreq(N, T)[:N//2]
        y1 = 2.0/N * np.abs(yf1[0:N//2])
        XfftIMFA_IMP[i] = x1
        YfftIMFA_IMP[i] = y1
        
        y2 = IMF7_IMP[i]
        yf2 = fft(y2)
        x2 = fftfreq(N, T)[:N//2]
        y2 = 2.0/N * np.abs(yf2[0:N//2])
        XfftIMFP_IMP[i] = x2
        YfftIMFP_IMP[i] = y2
        
        y3 = IMF8_IMP[i]
        yf3 = fft(y3)
        x3 = fftfreq(N, T)[:N//2]
        y3 = 2.0/N * np.abs(yf3[0:N//2])
        XfftIMFU_IMP[i] = x3
        YfftIMFU_IMP[i] = y3
        
        ####################################################
        # Valor Medido
        
        z0 = y0[5:60].argmax()
        w0 = x0[z0+5]
        ValorMedidoIMFO_IMP[i] = w0*60
        
        z1 = y1[5:60].argmax()
        w1 = x1[z1+5]
        ValorMedidoIMFA_IMP[i] = w1*60
        
        z2 = y2[5:60].argmax()
        w2 = x2[z2+5]
        ValorMedidoIMFP_IMP[i] = w2*60
        
        z3 = y3[5:60].argmax()
        w3 = x3[z3+5]
        ValorMedidoIMFU_IMP[i] = w3*60
        ####################################################

In [ ]:
%%time

## FFT PLOT
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

# Create figure
fig = make_subplots(rows=4, cols=1)
# Add all traces of pacient
for step in range(1,54):
    X = XfftIMFO_IMP[step]
    Y = YfftIMFO_IMP[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=1, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="gold")
for step in range(1,54):
    X = XfftIMFA_IMP[step]
    Y = YfftIMFA_IMP[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=2, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    X = XfftIMFP_IMP[step]
    Y = YfftIMFP_IMP[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=3, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="green")
for step in range(1,54):
    X = XfftIMFU_IMP[step]
    Y = YfftIMFU_IMP[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=4, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="blue") 

# Make 10th trace visible
fig.data[0].visible = True
fig.data[53].visible = True
fig.data[106].visible = True
fig.data[159].visible = True


# Create and add slider
steps = []
for i in range(53):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Padrão ouro dos pacientes: " + str(i+1)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+53] = True
    step["args"][0]["visible"][i+106] = True
    step["args"][0]["visible"][i+159] = True
    steps.append(step)

sliders = [dict(
    steps=steps
)]

# Update xaxis properties
fig.update_xaxes(title_text="frequencia (Hz)", row=4, col=1)

# Update yaxis properties
fig.update_yaxes(title_text="IMPEDANCIA", row=1, col=1)
fig.update_yaxes(title_text="IMF6", row=2, col=1)
fig.update_yaxes(title_text="IMF7", row=3, col=1)
fig.update_yaxes(title_text="IMF8", row=4, col=1)

fig.update_layout(
    title="FFT do sinal de Impedancia:",
    sliders=sliders,
    plot_bgcolor='rgba(0,0,0,0)',
    height=600
)

# Edit slider labels
fig['layout']['sliders'][0]['currentvalue']['prefix']='Paciente: '
for i in range(0,53):
    fig['layout']['sliders'][0]['steps'][i]['label']=i+1


fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_14'+'.html',auto_open=False)

In [ ]:
## PADRAO OURO
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

# Create figure
fig = make_subplots(rows=4, cols=1)

X = np.linspace(1, 1, 54)

# Add all traces of pacient
for i in range(1,54):
    X[i] = i
    
fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black",row=1, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMFO_IMP, name = "(x,y)", row=1, col=1,marker_color="gold", showlegend = False)
    
fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black", row=2, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMFA_IMP, name = "(x,y)", row=2, col=1,marker_color="red", showlegend = False)


fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black", row=3, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMFP_IMP, name = "(x,y)", row=3, col=1,marker_color="green", showlegend = False)

fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black", row=4, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMFU_IMP, name = "(x,y)", row=4, col=1, marker_color="blue", showlegend = False) 

# Make 10th trace visible
fig.data[0].visible = True

# Update xaxis properties
fig.update_xaxes(title_text="Respirações (rpm)", row=4, col=1)

# Update yaxis properties
fig.update_yaxes(title_text="IMPEDANCIA", row=1, col=1)
fig.update_yaxes(title_text="IMF6", row=2, col=1)
fig.update_yaxes(title_text="IMF7", row=3, col=1)
fig.update_yaxes(title_text="IMF8", row=4, col=1)

fig.update_layout(
    title="Valores da Frequencia respiratoria obtidos no sinal de Impedancia:",
    plot_bgcolor='rgba(0,0,0,0)',
    height=600
)

fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_15'+'.html',auto_open=False)

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

A = ValorOuro[1:54]
F = ValorMedidoIMFO_IMP[1:54]
G = ValorMedidoIMFA_IMP[1:54]
H = ValorMedidoIMFP_IMP[1:54]
I = ValorMedidoIMFU_IMP[1:54]
print(rmse(A, F))
print(rmse(A, G))
print(rmse(A, H))
print(rmse(A, I))

In [ ]:
%%time

#from numpy import empty
Sinal_Original = np.empty((54, 7500))
IMF1 = np.empty((54, 7500))
IMF2 = np.empty((54, 7500))
IMF3 = np.empty((54, 7500))
IMF4 = np.empty((54, 7500))
IMF5 = np.empty((54, 7500))
IMF6 = np.empty((54, 7500))
IMF7 = np.empty((54, 7500))
IMF8 = np.empty((54, 7500))

for i in range(54):
    
    if i == 0:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print(current_time)
    if i > 0 and i < 10:
        caminho5 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_0"+str(i)+"_Signals.csv"
        
        ###################################################
        # EEMD
        # Define signal
        signals = pd.read_csv(caminho5)
        s = signals[[' PLETH']]
        s = s.to_numpy()
        s_trans = s.transpose()
        Signal_PPG = s_trans[0]
        time = np.linspace(0, 1, 7500)

        # Assign max IMF on signal
        emd1 = EEMD()
        #emd1 = EEMD(MAX_ITERATION = 10)

        # Execute EEMD on signal
        #IMF = emd1.eemd(Signal_PPG,time, max_imf = 3)
        IMF = emd1.eemd(Signal_PPG[0:7500],time)

        Sinal_Original[i] = Signal_PPG[0:7500]
        IMF1[i] = IMF[0]
        IMF2[i] = IMF[1]
        IMF3[i] = IMF[2]
        IMF4[i] = IMF[3]
        IMF5[i] = IMF[4]
        IMF6[i] = IMF[5]
        IMF7[i] = IMF[6]
        IMF8[i] = IMF[7]

        ####################################################
                
    if i >= 10 and i<54:
        caminho5 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_"+str(i)+"_Signals.csv"
        
        ####################################################
        # EEMD
        # Define signal
        signals = pd.read_csv(caminho5)
        s = signals[[' PLETH']]
        s = s.to_numpy()
        s_trans = s.transpose()
        Signal_PPG = s_trans[0]
        time = np.linspace(0, 1, 7500)     

        # Assign max IMF on signal
        emd1 = EEMD()
        #emd1 = EEMD(MAX_ITERATION = 10)

        # Execute EEMD on signal
        #IMF = emd1.eemd(Signal_PPG,time, max_imf = 3)
        IMF = emd1.eemd(Signal_PPG[0:7500],time)

        Sinal_Original[i] = Signal_PPG[0:7500]
        IMF1[i] = IMF[0]
        IMF2[i] = IMF[1]
        IMF3[i] = IMF[2]
        IMF4[i] = IMF[3]
        IMF5[i] = IMF[4]
        IMF6[i] = IMF[5]
        IMF7[i] = IMF[6]
        IMF8[i] = IMF[7]

        ####################################################

In [ ]:
%%time
## IMF PLOT
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

X = np.linspace(0, 1, 2000)
# Create figure
fig = make_subplots(rows=9, cols=1)
# Add all traces of pacient

for step in range(1,54):
    Y = Sinal_Original[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=1, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="black")
for step in range(1,54):
    Y = IMF1[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=2, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF2[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=3, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF3[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=4, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF4[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=5, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF5[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=6, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF6[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=7, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF7[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=8, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF8[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=9, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")

# Make 10th trace visible
fig.data[0].visible = True
fig.data[53].visible = True
fig.data[106].visible = True
fig.data[159].visible = True
fig.data[212].visible = True
fig.data[265].visible = True
fig.data[318].visible = True
fig.data[371].visible = True
fig.data[424].visible = True

# Create and add slider
steps = []
for i in range(53):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Padrão ouro dos pacientes: " + str(i+1)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+53] = True
    step["args"][0]["visible"][i+106] = True
    step["args"][0]["visible"][i+159] = True
    step["args"][0]["visible"][i+212] = True
    step["args"][0]["visible"][i+265] = True
    step["args"][0]["visible"][i+318] = True
    step["args"][0]["visible"][i+371] = True
    step["args"][0]["visible"][i+424] = True

    steps.append(step)

sliders = [dict(   
    steps=steps
)]

# Update xaxis properties
fig.update_xaxes(title_text="segundos (s)", row=9, col=1)


# Update yaxis properties
fig.update_yaxes(title_text="PPG", row=1, col=1)
fig.update_yaxes(title_text="IMF1", row=2, col=1)
fig.update_yaxes(title_text="IMF2", row=3, col=1)
fig.update_yaxes(title_text="IMF3", row=4, col=1)
fig.update_yaxes(title_text="IMF4", row=5, col=1)
fig.update_yaxes(title_text="IMF5", row=6, col=1)
fig.update_yaxes(title_text="IMF6", row=7, col=1)
fig.update_yaxes(title_text="IMF7", row=8, col=1)
fig.update_yaxes(title_text="IMF8", row=9, col=1)

fig.update_layout(
    title="EEMD do sinal de PPG:",
    sliders=sliders,
    plot_bgcolor='rgba(0,0,0,0)',
    height=900
)

# Edit slider labels
fig['layout']['sliders'][0]['currentvalue']['prefix']='Paciente: '
for i in range(0,53):
    fig['layout']['sliders'][0]['steps'][i]['label']=i+1


fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_16'+'.html',auto_open=False)

In [ ]:
%%time

# Number of sample points
N = 7500
# sample spacing
T = 1.0 / 125
XfftIMF5 = np.empty((54, 3750))
YfftIMF5 = np.empty((54, 3750))
XfftIMF6 = np.empty((54, 3750))
YfftIMF6 = np.empty((54, 3750))
XfftIMF7 = np.empty((54, 3750))
YfftIMF7 = np.empty((54, 3750))
ValorMedidoIMF5 = np.array(range(54))
ValorMedidoIMF6 = np.array(range(54))
ValorMedidoIMF7 = np.array(range(54))

for i in range(54):
    
    if i == 0:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print(current_time)
    if i > 0 and i < 54:
        
        ####################################################
        # FFT

        y1 = IMF6[i]
        yf1 = fft(y1)
        x1 = fftfreq(N, T)[:N//2]
        y1 = 2.0/N * np.abs(yf1[0:N//2])
        XfftIMF5[i] = x1
        YfftIMF5[i] = y1
        
        y2 = IMF7[i]
        yf2 = fft(y2)
        x2 = fftfreq(N, T)[:N//2]
        y2 = 2.0/N * np.abs(yf2[0:N//2])
        XfftIMF6[i] = x2
        YfftIMF6[i] = y2
        
        y3 = IMF8[i]
        yf3 = fft(y3)
        x3 = fftfreq(N, T)[:N//2]
        y3 = 2.0/N * np.abs(yf3[0:N//2])
        XfftIMF7[i] = x3
        YfftIMF7[i] = y3
        ####################################################
        # Valor Medido
        
        z1 = y1[5:64].argmax()
        w1 = x1[z1+5]
        ValorMedidoIMF5[i] = w1*60
        
        z2 = y2[5:64].argmax()
        w2 = x2[z2+5]
        ValorMedidoIMF6[i] = w2*60
        
        z3 = y3[5:64].argmax()
        w3 = x3[z3+5]
        ValorMedidoIMF7[i] = w3*60
        ####################################################

In [ ]:
%%time

## FFT PLOT
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

# Create figure
fig = make_subplots(rows=3, cols=1)
# Add all traces of pacient
for step in range(1,54):
    X = XfftIMF5[step]
    Y = YfftIMF5[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=1, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    X = XfftIMF6[step]
    Y = YfftIMF6[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=2, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="green")
for step in range(1,54):
    X = XfftIMF7[step]
    Y = YfftIMF7[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=3, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="blue") 

# Make 10th trace visible
fig.data[0].visible = True
fig.data[53].visible = True
fig.data[106].visible = True


# Create and add slider
steps = []
for i in range(53):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Padrão ouro dos pacientes: " + str(i+1)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+53] = True
    step["args"][0]["visible"][i+106] = True
    steps.append(step)

sliders = [dict(
    steps=steps
)]

# Update xaxis properties
fig.update_xaxes(title_text="Frequencia (Hz)", row=3, col=1)


# Update yaxis properties
fig.update_yaxes(title_text="IMF6", row=1, col=1)
fig.update_yaxes(title_text="IMF7", row=2, col=1)
fig.update_yaxes(title_text="IMF8", row=3, col=1)

fig.update_layout(
    title="FFT do sinal de PPG:",
    sliders=sliders,
    plot_bgcolor='rgba(0,0,0,0)',
    height=600
)

# Edit slider labels
fig['layout']['sliders'][0]['currentvalue']['prefix']='Paciente: '
for i in range(0,53):
    fig['layout']['sliders'][0]['steps'][i]['label']=i+1


fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_17'+'.html',auto_open=False)

In [ ]:
## PADRAO OURO
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

# Create figure
fig = make_subplots(rows=3, cols=1)

X = np.linspace(1, 1, 54)

# Add all traces of pacient
for i in range(1,54):
    X[i] = i
    

fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black",row=1, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMF5, name = "(x,y)", row=1, col=1,marker_color="red", showlegend = False)


fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black",row=2, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMF6,name = "(x,y)", row=2, col=1,marker_color="green", showlegend = False)

fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black",row=3, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMF7, name = "(x,y)", row=3, col=1, marker_color="blue", showlegend = False)



  

# Make 10th trace visible
fig.data[0].visible = True

# Update xaxis properties
fig.update_xaxes(title_text="Respirações (rpm)", row=3, col=1)


# Update yaxis properties
fig.update_yaxes(title_text="IMF6", row=1, col=1)
fig.update_yaxes(title_text="IMF7", row=2, col=1)
fig.update_yaxes(title_text="IMF8", row=3, col=1)


fig.update_layout(
    title="Valores da Frequencia respiratoria obtidos no sinal de PPG:",
    plot_bgcolor='rgba(0,0,0,0)',
    height=600
)

fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_18'+'.html',auto_open=False)

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

A = ValorOuro[1:54]
F = ValorMedidoIMF5[1:54]
G = ValorMedidoIMF6[1:54]
H = ValorMedidoIMF7[1:54]
print(rmse(A, F))
print(rmse(A, G))
print(rmse(A, H))
